# Loading data

In [75]:
from numbers import Number
from collections import defaultdict

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

import sina.datastores.sql as sina_sql
import sina.utils
from sina.datastore import create_datastore
from sina.visualization import Visualizer
import math
import kosh

%matplotlib notebook

# Initialization
database = './data/baseline_output.sqlite'
target_type = "csv_rec"
datastore = kosh.connect(database)
# recs = datastore.records
# vis = Visualizer(datastore)
print("Sina and Kosh are ready!")

Sina and Kosh are ready!


# Adding Data to Records

In [80]:
def RMSE(x_true, x_pred):
    summation = 0
    for xt, xp in zip(x_true, x_pred):
        summation += (xt-xp)**2
    mean = summation/len(x_true)
    rmse = math.sqrt(mean)
    return rmse


def diff(x_true, x_pred):
    dif = []
    for xt, xp in zip(x_true, x_pred):
        dif.append(xt-xp)
    return dif

val = list(datastore.find(id_pool = '47bcda_0'))[0] #need to convert to list then extract
# Printing Attributes and Features
print('Attributes:')
print('\t',val.list_attributes())
print('\n')
print('Features Sets:')
print('\t',val.list_features())

# Validation Data
x_true = val['physics_cycle_series/x_pos'][:]
y_true = val['physics_cycle_series/y_pos'][:]
z_true = val['physics_cycle_series/z_pos'][:]
time = val['physics_cycle_series/time'][:]

# Adding RMSE and Diff data to each record
for dataset in datastore.find(): # Each record is now a dataset
    print(f"id: {dataset.id}")
    x_pred = dataset['physics_cycle_series/x_pos'][:]
    y_pred = dataset['physics_cycle_series/y_pos'][:]
    z_pred = dataset['physics_cycle_series/z_pos'][:]

    x_diff = diff(x_true, x_pred)
    x_rmse = RMSE(x_true, x_pred)
    y_diff = diff(y_true, y_pred)
    y_rmse = RMSE(y_true, y_pred)
    z_diff = diff(z_true, z_pred)
    z_rmse = RMSE(z_true, z_pred)

    print(f"\tx_rmse: {x_rmse}")
    print(f"\ty_rmse: {y_rmse}")
    print(f"\tz_rmse: {z_rmse}")

    setattr(dataset, 'x_rmse', x_rmse)
    dataset.add_curve(x_diff,'x_diff', 'physics_cycle_series')
    setattr(dataset, 'y_rmse', y_rmse)
    dataset.add_curve(y_diff,'y_diff', 'physics_cycle_series')
    setattr(dataset, 'z_rmse', z_rmse)
    dataset.add_curve(z_diff,'z_diff', 'physics_cycle_series')

    recs.update(rec)  # need to update or else won't save!!!!!

Attributes:
	 ['box_side_length', 'gravity', 'group_id', 'id', 'num_bounces', 'x_pos_final', 'x_pos_initial', 'x_vel_final', 'x_vel_initial', 'y_pos_final', 'y_pos_initial', 'y_vel_final', 'y_vel_initial', 'z_pos_final', 'z_pos_initial', 'z_vel_final', 'z_vel_initial']


Features Sets:
	 ['physics_cycle_series', 'physics_cycle_series/cycle', 'physics_cycle_series/time', 'physics_cycle_series/x_pos', 'physics_cycle_series/y_pos', 'physics_cycle_series/z_pos']
id: 47bcda_3
	x_rmse: 0.19772673369471488
	y_rmse: 0.11232115336574376
	z_rmse: 1.430031261314339


AttributeError: Object 47bcda_3 does not have add_curve attribute

# Parameter Table

In [ ]:
# Printing parameter table in Markdown
header = 1
for rec in recs.get_all():

    if header == 1:
        print('| rec.id | x_pos_initial | y_pos_initial | z_pos_initial | x_vel_initial | y_vel_initial | z_vel_initial |')
        print('| --- | --- | --- | --- | --- | --- | --- |')
        header = 0
    print('|', rec.id,
          '|', rec.data['x_pos_initial']['value'],
          '|', rec.data['y_pos_initial']['value'],
          '|', rec.data['z_pos_initial']['value'],
          '|', rec.data['x_vel_initial']['value'],
          '|', rec.data['y_vel_initial']['value'],
          '|', rec.data['z_vel_initial']['value'],
          '|'
          )

# Plotting QoIs and their Diff


In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))


_ = vis.create_line_plot(fig=fig, ax=ax[0, 0], x="time", y="x_pos", title="{y_name}")
_ = vis.create_line_plot(fig=fig, ax=ax[0, 1], x="time", y="y_pos", title="{y_name}")
_ = vis.create_line_plot(fig=fig, ax=ax[0, 2], x="time", y="z_pos", title="{y_name}")

_ = vis.create_line_plot(fig=fig, ax=ax[1, 0], x="time", y="x_diff", title="{y_name}")
_ = vis.create_line_plot(fig=fig, ax=ax[1, 1], x="time", y="y_diff", title="{y_name}")
_ = vis.create_line_plot(fig=fig, ax=ax[1, 2], x="time", y="z_diff", title="{y_name}")

fig.savefig("./images/QoIs.png")

# RMSE Plots

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 10))

header = 1
for rec in recs.get_all():
    ax[0].bar(x=rec.id, height=rec.data['x_rmse']['value'])
    ax[1].bar(x=rec.id, height=rec.data['y_rmse']['value'])
    ax[2].bar(x=rec.id, height=rec.data['z_rmse']['value'])

    if header == 1:
        print('| rec.id | x_rmse | y_rmse | z_rmse | Mean RMSE |')
        print('| --- | --- | --- | --- | --- |')
        header = 0
    print('|', rec.id,
          '|', round(rec.data['x_rmse']['value'], 2),
          '|', round(rec.data['y_rmse']['value'], 2),
          '|', round(rec.data['z_rmse']['value'], 2),
          '|', round((rec.data['x_rmse']['value']+rec.data['y_rmse']['value']+rec.data['z_rmse']['value'])/3, 2),
          '|'
          )

ax[0].tick_params(rotation=45)
ax[1].tick_params(rotation=45)
ax[2].tick_params(rotation=45)

ax[0].set_title('x_rmse')
ax[1].set_title('y_rmse')
ax[2].set_title('z_rmse')

ax[0].set_ylabel('x_rmse')
ax[1].set_ylabel('y_rmse')
ax[2].set_ylabel('z_rmse')

fig.savefig("./images/RMSE.png")